## Pointing error ECAM analysis using my plate solving

In [1]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, AltAz, EarthLocation
from astropy.time import Time
import os
import subprocess
import glob
#from astroquery.astrometry_net import AstrometryNet
from astropy.io import fits
import sys
from termcolor import colored

### Platesolving the images

In [2]:
# Input and output directories
fits_dir = "/home/alberte2/Sample_docs/ECAM_images_analysis-02-04-2025/"
output_dir = "/home/alberte2/Sample_docs/ECAM_images_analysis-02-04-2025/solved/"
os.makedirs(output_dir, exist_ok=True)

# Pixel scale range (arcsec per pixel)
scale_low = 0.21
scale_high = 0.22  # True value is 0.215

# Get a list of FITS files in the input directory
fits_files = glob.glob(os.path.join(fits_dir, "*.fits"))

# Loop through the FITS files
for fits_path in fits_files:
    file_name = os.path.basename(fits_path)
    output_name = file_name.replace(".fits", ".solved.fits")

    # Open FITS file
    with fits.open(fits_path) as hdul:
        header = hdul[0].header
        ra_approx = header['HIERARCH OGE TEL TARG ALPHA INST']  # Fetch RA from header
        dec_approx = header['HIERARCH OGE TEL TARG DELTA INST']  # Fetch Dec from header

        try:
            # Run solve-field for each FITS file with specified parameters
            result = subprocess.run(
                [
                    "ssh", "glsastro",
                    f"cd {output_dir} &&",  # Ensure working directory
                    "solve-field",
                    #"--verbose",  # Enables verbose output (chatty)
                    "--overwrite",  # Overwrite existing output files
                    "--no-verify",  # Skips verification of FITS headers
                    #"--no-fits2fits",  # Avoid creating intermediate FITS files. Looks like this argument is deprecated.
                    f"--ra {ra_approx}",  # Approximate RA (Right Ascension)
                    f"--dec {dec_approx}",  # Approximate Dec (Declination)
                    "--scale-units", "arcsecperpix",  # Scale units
                    f"--scale-low {scale_low}",  # Minimum pixel scale
                    f"--scale-high {scale_high}",  # Maximum pixel scale
                    "--crpix-x","2121", #set the WCS reference point to the given position
                    "--crpix-y","2121.5", #set the WCS reference point to the given position
                    "--radius", "3",  # Search radius
                    "--depth", "50", #controls number of features to be considered during solving. Higher values increases accuracy but takes longer time.
                    "--objs", "100", #maximum number of objects used for solving
                    "-z 4",
                    "--no-plots",  # Suppress plot outputs
                    f"--dir {output_dir}",  # Directory for output files
                    f"--new-fits {output_name}",  # Name of the new FITS file
                    fits_path  # The FITS file to be solved
                ],
                check=True,  # Raise an exception if the command fails
                stdout=subprocess.PIPE,  # Capture stdout
                stderr=subprocess.PIPE,  # Capture stderr
                text=True  # Ensure text output
            )
            # Display output
            print(colored("*******************************************", "green"))
            print(" ")
            print(colored(f"Output for {file_name}:", "blue", attrs=["bold"]))
            print(result.stdout)  # Show standard output
            print(colored("*******************************************", "green"))
            print(colored(f"SOLVED: {file_name}", "green", attrs=["bold"]))

        except subprocess.CalledProcessError as e:
            print(colored("*******************************************", "red"))
            print(colored(f"Failed to solve: {file_name}.", "red", attrs=["bold"]))
            print(f"Error: {e.stderr}")
            print(colored("*******************************************", "red"))
            print(" ")


*******************************************
 
Output for ECAM.2025-04-02T10:03:41.000.fits:
Reading input file 1 of 1: "/home/alberte2/Sample_docs/ECAM_images_analysis-02-04-2025/ECAM.2025-04-02T10:03:41.000.fits"...
Extracting sources...
Downsampling by 4...
simplexy: found 906 sources.
Solving...
Failed to add index "/usr/share/astrometry/index-tycho2-19.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-18.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-17.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-16.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-15.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-14.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-13.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-12.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-11.bigendian.fits".
Failed to add in

### Process solved files

In [3]:
# Observatory location for CORALIE (example: La Silla)
obs_location = EarthLocation(lon=-70.732951945082, lat=-29.259497822801, height=2378.0)

# Step 2: Process FITS Files
def process_fits_images(output_dir):
    pointing_data = []

    for file_name in os.listdir(output_dir):
        if file_name.endswith('.solved.fits'):
            fits_path = os.path.join(output_dir, file_name)

            try:
                # Open FITS file
                with fits.open(fits_path) as hdul:
                    header = hdul[0].header
                    w = WCS(header)
                    #print('wcs: ', w) 
                    obs_time = Time(float(header.get('HIERARCH OGE TEL TARG TUNIX INST')), format = 'unix')

                    #print("Obs time: ", obs_time)

                    # Extract expected azimuth and elevation
                    az_expected = header.get('HIERARCH OGE TEL TARG AZI INST', None) % 360 #to avoid negative angles but not a problem for tpoint in principle
                    el_expected = header.get('HIERARCH OGE TEL TARG ELE INST', None)

                    # Fiber pixel coordinates
                    x_ref = 2121 #header.get('HIERARCH OGE DET PIXELREF X', None)
                    y_ref = 2121.5 #header.get('HIERARCH OGE DET PIXELREF Y', None)
                    #print("Ref pixel: ", x_ref, y_ref)

                    # Check for missing data
                    if az_expected is None or el_expected is None:
                        print(f"Missing AZIMUTH/ELEVATION in {file_name}. Skipping...")
                        continue
                    if x_ref is None or y_ref is None:
                        print(f"Missing X_ref/Y_ref in {file_name}. Skipping...")
                        continue


                    
                    #Converting the ra dec in the header to horizontal coords i.e skipping the ones mentioned in the header
                    #sky_coord = SkyCoord(
                    #    ra=header.get('RA', None), dec=header.get('DEC', None),
                    #    frame="icrs", unit="deg"
                    
                    #altaz_frame = AltAz(location=obs_location, obstime=obs_time)
                    #altaz_coord = sky_coord.transform_to(altaz_frame)

                    #az_expected = altaz_coord.az.deg #+ 180) % 360
                    #el_expected = altaz_coord.alt.deg
                    #print("az actual: ", az_actual)



                    
                    # Convert pixel coordinates to RA/Dec using WCS
                    pixel_world_coords = w.pixel_to_world(x_ref, y_ref)
                    #print(fiber_world_coords)
                    # Transform RA/Dec to Azimuth/Elevation
                    sky_coord = SkyCoord(
                        ra=pixel_world_coords.ra, dec=pixel_world_coords.dec,
                        frame="icrs", unit="deg"
                    )
                    altaz_frame = AltAz(location=obs_location, obstime=obs_time)
                    altaz_coord = sky_coord.transform_to(altaz_frame)

                    az_actual = (altaz_coord.az.deg + 180) % 360
                    el_actual = altaz_coord.alt.deg
                    #print("az actual: ", az_actual)

                    # Save data
                    pointing_data.append({
                        #'file': file_name,
                        'az_expected': az_expected,
                        'el_expected': el_expected,
                        'az_actual': az_actual,
                        'el_actual': el_actual,
                    })
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

    return pointing_data


In [6]:
# Step 3: Generate TPoint-Compatible Data File
def create_pointing_data_file(data, pointing_file):
    try:
        with open(pointing_file, "w+") as out:
            out.write("! Pointing Data for ECAM\n")
            out.write(":NODA\n")
            out.write("-29 15 24.0\n\n")  # Latitude

            for entry in data:
                out.write(f"{entry['az_expected']} {entry['el_expected']} {entry['az_actual']} {entry['el_actual']}\n")
            
            out.write("END\n")
        print(f"Pointing data file created at {pointing_file}")
    except Exception as e:
        print(f"Error creating pointing data file: {e}")


# Step 4: Compute Pointing Model
def compute_pointing_model(data_file, output_dir, model_name):
    os.makedirs(output_dir, exist_ok=True)
    tpoint_script = os.path.join(output_dir, "TPoint.dat")
    
    try:
        with open(tpoint_script, "w+") as f:
            f.write(f"indat {data_file}\n")
            f.write("USE AN AW IA IE CA TX HZSZ2 ECES\n")
            f.write("fit\n")
            f.write("call a9\n")  # creates 9 auto plots
            f.write("spawn sleep 8\n")  # Adds a delay for plotting
            f.write(f"OUTMOD {model_name}.tp\n")
            f.write(f"GC P {model_name}.ps\n")
            f.write("end\n")

        os.chdir(output_dir)
        subprocess.run(f"tpoint < {tpoint_script}", shell=True, check=True)
        print(colored("Pointing model computed successfully.", "green"))
    except Exception as e:
        print(colored(f"Error computing pointing model: {e}", "red"))


In [7]:
# Example Execution
pointing_data_file = f"{fits_dir}Ecam_astrometry_pm-02-04-2025.dat"  # Replace with desired output file
# Process data
pointing_data = process_fits_images(output_dir)
# Prepare data file for TPoint analysis (no corrections applied)
create_pointing_data_file(pointing_data, pointing_data_file)
# Compute pointing model (if needed for analysis)
compute_pointing_model(pointing_data_file,fits_dir, "Ecam_pointing_model-02-04-2025")

Pointing data file created at /home/alberte2/Sample_docs/ECAM_images_analysis-02-04-2025/Ecam_astrometry_pm-02-04-2025.dat
running /opt/t4/beta/src/simond/tpoint/bin/tpt / / /dev/null/ /opt/t4/beta/src/simond/tpoint/etc/tpoint/tpoint.ini
    initialization file: /opt/t4/beta/src/simond/tpoint/etc/tpoint/tpoint.ini
    log file: /dev/null/













+ - - - - - - - - - - - - - - - - - - - - +
|                 TPOINT                  |
|   Telescope Pointing Analysis System    |
|              Version 5.2-1              |
+ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ +

There are 54 standard pointing terms.
Reading procedures from file /opt/t4/beta/src/simond/tpoint/etc/tpoint/procs.dat ...
244 lines input.
Reading star catalogue entries from file /opt/t4/beta/src/simond/tpoint/etc/tpoint/stars.dat ...
209 stars input.

TPOINT ready for use:  type HELP for assistance, END to quit.

* ! Pointing Data for ECAM
:NODA
-29 15 24.0

178.336563 26.4570599 178.37550691302692 26.34312272733092
17

### Other important things to include in the above script are plots of the pointing error in az and ele vs date, the pointing infomation as I did for the other ones, polar plots of the grid and possibly other important points.